In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
pd.set_option("display.max_colwidth", 500)
import requests
from bs4 import BeautifulSoup
from time import sleep
import random
import datetime


In [2]:
#画像のJs コード
def save_binary(url, filepath):
    js = """
    var getBinaryResourceText = function(url) {
        var req = new XMLHttpRequest();
        req.open('GET', url, false);
        req.overrideMimeType('text/plain; charset=x-user-defined');
        req.send(null);
        if (req.status != 200) return '';

        var filestream = req.responseText;
        var bytes = [];
        for (var i = 0; i < filestream.length; i++){
            bytes[i] = filestream.charCodeAt(i) & 0xff;
        }

        return bytes;
    }
    """
    js += "return getBinaryResourceText(\"{url}\");".format(url=url)

    data_bytes = driver.execute_script(js)
    with open(filepath, 'wb') as bin_out:
        bin_out.write(bytes(data_bytes))


capabilities = {
    'chromeOptions': {
        # 'androidPackage': 'com.android.chrome',
    }
}

In [3]:
#　集計の関数
def profile_data(user_html):
    user_data = pd.DataFrame(columns=['userid', 'name', 'age_area','regist_date','reported',
                                      'msg_cnt','write_board','profile','height','weight',
                                      'work','drink','tobacco','marrege','types',
                                      'access_date']
                            )
    userid = user_html.find('div',{'class':'Box01 profile'})['userid']
    name= user_html.find('span',{'class':'font_lll font_red'}).text
    age_area = user_html.find('ul',{'class':'situation'}).find('li').text
    regist_date = user_html.find('ul',{'class':'statistics'}).find_all('li')[0].text
    reported = user_html.find('ul',{'class':'statistics'}).find_all('li')[1].text

    msg_cnt = user_html.find('ul',{'class':'statistics'}).find_all('li')[2].text.split(':')[1]
    write_board = user_html.find('ul',{'class':'statistics'}).find_all('li')[3].text.split(':')[1]
    profile = user_html.find('div',{'class':'BoxGray'}).text
    height = user_html.find('div',{'class':'table03'}).find_all('dd')[0].text
    weight = user_html.find('div',{'class':'table03'}).find_all('dd')[1].text
    
    work = user_html.find('div',{'class':'table03'}).find_all('dd')[2].text
    drink = user_html.find('div',{'class':'table03'}).find_all('dd')[3].text
    tobacco = user_html.find('div',{'class':'table03'}).find_all('dd')[4].text
    marrege = user_html.find('div',{'class':'table03'}).find_all('dd')[5].text
    types = user_html.find('div',{'class':'table03'}).find_all('dd')[6].text
    
    access_date = datetime.date.today()
    
    series = pd.Series([userid, name, age_area,regist_date,reported,
                        msg_cnt,write_board,profile,height,weight,
                        work,drink,tobacco,marrege,types,
                        access_date], 
                       index=user_data.columns
                      )
    
    user_data = user_data.append(series,ignore_index = True)
    return user_data

In [4]:
##今回はGoogleChromeを使用
driver = webdriver.Chrome("C:\webdriver\chromedriver.exe")
id = "taka.happy@me.com"
password = "ssasdf1234["

url = "https://sugardaddy.jp/login"
driver.get(url)
sleep(4)

In [5]:
#ログイン
driver.find_element_by_name("email").send_keys(id)
driver.find_element_by_name("pass").send_keys(password)
sleep(2)
driver.find_element_by_class_name('btn_black').send_keys(Keys.ENTER)
user_detail = pd.DataFrame(columns=['userid', 'name', 'age_area','regist_date','reported','msg_cnt',
                               'write_board','profile','height','weight','work','drink','tobacco',
                               'marrege','types','access_date']
                          )

In [6]:
user_data = pd.DataFrame(columns=['userid', 'name', 'age_area','regist_date','reported','msg_cnt',
                               'write_board','profile','height','weight','work','drink','tobacco',
                               'marrege','types','access_date']
                     )

In [7]:
###　繰り返し場所

In [8]:
#　集計条件
start_url = "https://sugardaddy.jp/search"
driver.get(start_url)
sleep(2)

In [9]:
serch = BeautifulSoup(driver.page_source,"html.parser")
list= serch.find_all('div',{'class':'BoxPhotoProfile BoxGray clearfix'})
user_list = []
for i in list:
    url = "https://sugardaddy.jp/profile?id=" + i['userid']
    user_list.append(url)

In [10]:
%%time
user_page = []
for i in user_list:
    driver.get(i)
    user_url = BeautifulSoup(driver.page_source,"lxml")
    user_page.append(user_url)
    user_detail = user_detail.append(profile_data(user_url))
    userid = user_url.find('div',{'class':'Box01 profile'})['userid']
    photos = user_url.find_all('img',{'class':'popup'})
    n = 0
    for j in photos:
        try:
            img_url = ('https://sugardaddy.jp' + j['src'])
            file_name = '../img/' + str(userid) + '_image_' + str(n) + '.jpeg'
            n += 1
            save_binary(img_url, file_name)
            sleep(0.3 + 3*random.random())        
        except:
            None

    sleep(1 + 1*random.random())
    driver.get(start_url)
    sleep(1 + 3*random.random())

Wall time: 3min 46s


In [11]:
save_date = '../user_detail/' + datetime.datetime.now().strftime("%Y%m%d_%H%M") + '.csv'
user_detail.to_csv(save_date)

In [12]:
path = os.getcwd()
path

'C:\\Users\\Masuda Takashi\\python_code\\web_scraping'